# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.0203689
 0.548397 
 0.790404 
 0.249931 
 0.968914 
 0.650478 
 0.168058 
 0.103042 
 0.945138 
 0.327558 
 0.235755 
 0.714719 
 0.177021 
 ⋮        
 0.273238 
 0.90644  
 0.863469 
 0.53412  
 0.86914  
 0.234851 
 0.354912 
 0.90399  
 0.852363 
 0.874803 
 0.636237 
 0.748633 

In [2]:
sum(a)   

4.998313529595904e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [3]:
#Pkg.add("BenchmarkTools")

In [4]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [5]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [6]:
c_sum(a)

4.998313529595609e6

In [7]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [8]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [9]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [10]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.826 ms (0.00% GC)
  median time:      7.870 ms (0.00% GC)
  mean time:        7.898 ms (0.00% GC)
  maximum time:     8.443 ms (0.00% GC)
  --------------
  samples:          633
  evals/sample:     1

In [11]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 7.826267 msec


In [12]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 7.82627

In [13]:
using Plots
gr()

Plots.GRBackend()

In [14]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7.82 
 
 
 7.84 
 
 
 7.86 
 
 
 7.88 
 
 
 7.90 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [15]:
#Pkg.add("PyCall")

In [16]:
using PyCall

In [17]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [18]:
pysum(a)

4.998313529595609e6

In [19]:
pysum(a) ≈ sum(a)

true

In [20]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     64.827 ms (0.00% GC)
  median time:      65.297 ms (0.00% GC)
  mean time:        65.354 ms (0.00% GC)
  maximum time:     66.372 ms (0.00% GC)
  --------------
  samples:          77
  evals/sample:     1

In [21]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 7.82627
  "Python built-in" => 64.8268

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [22]:
using Conda 
#Conda.add("numpy")

In [23]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     4.493 ms (0.00% GC)
  median time:      4.543 ms (0.00% GC)
  mean time:        4.573 ms (0.00% GC)
  maximum time:     5.611 ms (0.00% GC)
  --------------
  samples:          1090
  evals/sample:     1

In [24]:
numpy_sum(apy_list) # python thing

4.998313529595904e6

In [25]:
numpy_sum(apy_list) ≈ sum(a)

true

In [26]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 7.82627
  "Python numpy"    => 4.49278
  "Python built-in" => 64.8268

# 4. Python, hand-written 

In [27]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x13ecf1b90>

In [28]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     416.654 ms (0.00% GC)
  median time:      417.815 ms (0.00% GC)
  mean time:        418.017 ms (0.00% GC)
  maximum time:     419.883 ms (0.00% GC)
  --------------
  samples:          12
  evals/sample:     1

In [29]:
sum_py(apy_list)

4.998313529595609e6

In [30]:
sum_py(apy_list) ≈ sum(a)

true

In [31]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 7.82627
  "Python numpy"        => 4.49278
  "Python hand-written" => 416.654
  "Python built-in"     => 64.8268

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [32]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [33]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.358 ms (0.00% GC)
  median time:      4.413 ms (0.00% GC)
  mean time:        4.461 ms (0.00% GC)
  maximum time:     5.452 ms (0.00% GC)
  --------------
  samples:          1118
  evals/sample:     1

In [34]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 7.82627
  "Python numpy"        => 4.49278
  "Python hand-written" => 416.654
  "Python built-in"     => 64.8268
  "Julia built-in"      => 4.35796

# 6. Julia (hand-written) 

In [35]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [36]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.845 ms (0.00% GC)
  median time:      7.907 ms (0.00% GC)
  mean time:        8.012 ms (0.00% GC)
  maximum time:     9.504 ms (0.00% GC)
  --------------
  samples:          624
  evals/sample:     1

In [37]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 7.82627
  "Python numpy"        => 4.49278
  "Julia hand-written"  => 7.84543
  "Python hand-written" => 416.654
  "Python built-in"     => 64.8268
  "Julia built-in"      => 4.35796

# Summary

In [38]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C........................7.8
Julia built-in...........4.4
Julia hand-written.......7.8
Python built-in.........64.8
Python hand-written....416.7
Python numpy.............4.5


In [39]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............4.36
Python numpy..............4.49
C.........................7.83
Julia hand-written........7.85
Python built-in..........64.83
Python hand-written.....416.65
